In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("../data/raw/INTERRUPCAO_CARGA.csv", sep=";")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/INTERRUPCAO_CARGA.csv'

In [ ]:
df.info()

In [ ]:
# Célula de Limpeza e Filtragem (VERSÃO CORRIGIDA PARA NOTEBOOK 01)

# Converter a coluna de data para o formato datetime do pandas
df['timestamp'] = pd.to_datetime(df['din_interrupcaocarga'])

# Filtrar o dataset para manter apenas os eventos de Goiás
df_goias = df[df['id_estado'] == 'GO'].copy()

df_goias = df_goias[df_goias['timestamp'].dt.year == 2024].copy()

# -------------------------------------------

# Definir o timestamp como o índice do DataFrame para facilitar a agregação
df_goias.set_index('timestamp', inplace=True)

print("Dados de Interrupção filtrados para Goiás e para o ano de 2025.")
display(df_goias.head())

In [ ]:
deficit_diario = df_goias['val_energianaosuprida_mwh'].resample('D').sum()
deficit_diario = deficit_diario.fillna(0)
df_diario = deficit_diario.to_frame(name='deficit_diario_mwh')

print("Tabela de déficit diário para Goiás:")
display(df_diario.head()) # <-- Queremos ver a saída desta linha

In [ ]:
# Calcular os limites apenas nos dias em que HOUVE algum déficit
dias_com_deficit = df_diario[df_diario['deficit_diario_mwh'] > 0]

# Limite para Risco Alto: Percentil 99 (o 1% pior dos eventos)
limite_alto = dias_com_deficit['deficit_diario_mwh'].quantile(0.99)

# Limite para Risco Médio: Percentil 95 (os 5% piores dos eventos)
limite_medio = dias_com_deficit['deficit_diario_mwh'].quantile(0.95)

print(f"Limite para Risco Médio (P95): {limite_medio:.2f} MWh")
print(f"Limite para Risco Alto (P99): {limite_alto:.2f} MWh")


# Função para aplicar a classificação
def classificar_risco(deficit):
    if deficit >= limite_alto:
        return 'alto'
    elif deficit >= limite_medio:
        return 'medio'
    else:
        # Dias com déficit zero ou pequeno são classificados como 'baixo'
        return 'baixo'

# Criar a coluna final da nossa variável alvo
df_diario['nivel_risco'] = df_diario['deficit_diario_mwh'].apply(classificar_risco)

## Desbalanceamento da classe

Os dados são muito desbalanceados, sendo a maioria dos níveis de risco baixos.

In [ ]:
# Verificar a distribuição das classes de risco
print("Distribuição das classes de risco:")
print(df_diario['nivel_risco'].value_counts())

# Plotar os déficits diários para visualizar os eventos
plt.figure(figsize=(15,7))
plt.title('Energia Não Suprida (Déficit) por Dia em Goiás')
df_diario['deficit_diario_mwh'].plot()
plt.ylabel('Déficit (MWh)')
plt.show()

In [ ]:
# Adicionar este código para um novo gráfico
plt.figure()
plt.title('Déficit Diário em Goiás (Escala Log)')
ax = df_diario['deficit_diario_mwh'].plot(logy=True, style='.', alpha=0.5)
ax.set_ylabel("Déficit (MWh) - Escala Log")
plt.show()

In [ ]:
# Adicionar no final do notebook
output_path = '../data/processed/target_variable_daily.parquet'
df_diario.to_parquet(output_path)
print(f"DataFrame final salvo em: {output_path}")